<a href="https://colab.research.google.com/github/UmarFarooqui/Springboard/blob/master/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pprint import pprint
import json
from pathlib import Path
import pandas as pd

In [25]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# Uploaded the dataset to google drive. Checking.
!ls "/content/drive/My Drive/data"

codesearchnet_python.zip


In [28]:
!unzip "/content/drive/My Drive/data/codesearchnet_python.zip"

Archive:  /content/drive/My Drive/data/codesearchnet_python.zip
replace python/final/jsonl/train/python_train_9.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_12.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_10.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_0.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_6.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_2.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_4.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_8.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_11.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace python/final/jsonl/train/python_train_5.jso

In [0]:
# Exploring the full dataset
python_files = sorted(Path('python/').glob('**/*.gz'))

In [36]:
python_files

[PosixPath('python/final/jsonl/test/python_test_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_0.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_1.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_10.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_11.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_12.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_13.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_2.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_3.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_4.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_5.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_6.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_7.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_8.jsonl.gz'),
 PosixPath('python/final/jsonl/train/python_train_9.jsonl.gz'),
 PosixPath('python/final/jsonl/valid/p

In [37]:
print(f'Total number of files: {len(python_files)}')

Total number of files: 16


In [0]:
columns_list = ['repo', 'path', 'url', 'func_name', 'code', 'code_tokens', 'docstring', 'docstring_tokens']

In [0]:
def jsonl_list_to_dataframe(file_list, columns=columns_list):
  """Load a list of jsonl.gz files into a pandas dataframe."""
  return pd.concat([pd.read_json(f, orient='records', compression='gzip', lines=True)[columns] for f in file_list], sort=False) 

In [0]:
pydf = jsonl_list_to_dataframe(all_files)

In [60]:
pydf.head(3)

,repo,path,url,func_name,code,code_tokens,docstring,docstring_tokens
0,soimort/you-get,src/you_get/extractors/youtube.py,https://github.com/soimort/you-get/blob/b746ac...,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]"
1,soimort/you-get,src/you_get/extractors/miomio.py,https://github.com/soimort/you-get/blob/b746ac...,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,..."
2,soimort/you-get,src/you_get/extractors/fc2video.py,https://github.com/soimort/you-get/blob/b746ac...,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co..."


In [0]:
# The columns which will be used for our purpose would be func_name, code, code_tokens, docstring, docstring_tokens